# Scraping Wikipedia Page
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(website_url,'lxml')

### Scraping table from wikipedia

In [3]:
table = soup.find('table',{'class':'wikitable sortable'})

In [4]:
A=[]
B=[]
C=[]

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))


In [5]:
df=pd.DataFrame()
df['Postcode']=A
df['Borough']=B
df['Neighbourhood']=C

In [6]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Ignoring cells with not assigned values

In [7]:
df1=df[df.Borough != 'Not assigned']

In [8]:
df1=df1.reset_index(drop=True)

In [9]:
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


### Merging Neighbourhood with same Postcode

In [10]:
index_cols = df1.columns.tolist()
index_cols.remove("Neighbourhood")
df2=df1.groupby(index_cols,sort=False)["Neighbourhood"].apply(list)
df2=df2.reset_index()

In [11]:
Neighbourhood=[]
for i in range(103):
    s=df2['Neighbourhood'][i]
    line = ', '.join([str(elem) for elem in s]).replace('\n','')
    Neighbourhood.append(line)

In [12]:
df2['Neighbourhood']=Neighbourhood

In [13]:
df2.head(6)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Not assigned


### 'Not assigned' Neighbourhood to corresponding Borough

In [14]:
df2.Neighbourhood[df2.Neighbourhood == 'Not assigned']=df2.Borough[df2.Neighbourhood == 'Not assigned']

### Required Dataframe and it's Shape

In [15]:
df2.head(6)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park


In [16]:
df2.shape

(103, 3)